In [60]:
import pandas as pd 
import seaborn as sns
import matplotlib as plt
import basedosdados as bd
sns.set_theme(style="whitegrid")

In [69]:
#conda update -n base -c defaults conda
!pip install --upgrade pandas


     --------------------------------------- 11.6/11.6 MB 13.9 MB/s eta 0:00:00
     --------------------------------------- 16.6/16.6 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 2.0.1 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.1 which is incompatible.
basedosdados 1.6.11 requires pandas<2.0.0,>=1.3.5, but you have pandas 2.2.2 which is incompatible.


In [ ]:

# Instalar o pacote da basedosdados, caso seja a primeira vez a rodar o script
#! pip install basedosdados

In [61]:
# Para carregar e salvar os dados 

query = """

SELECT ano, mes, sigla_uf, saldo_movimentacao, id_municipio, cnae_2_secao, categoria, cnae_2_subclasse, cbo_2002, grau_instrucao, sexo, salario_mensal, indicador_trabalho_intermitente,
indicador_trabalho_parcial,   FROM `basedosdados.br_me_caged.microdados_movimentacao` WHERE cnae_2_subclasse  IN (
      --esssas duas subclasses são da divisão 50 TRANSPORTE AQUAVIÁRIO
      '5231101', --Administração da infraestrutura portuária
      '5231102', --Atividades do operador portuário
      --esssas outras são novas
      '5011401', --Transporte marítimo de cabotagem - carga
      '5011402', --Transporte marítimo de cabotagem - passageiros
      '5021101', --Transporte por navegação interior de carga, municipal, exceto travessia
      '5021102', --Transporte por navegação interior de carga, intermunicipal, interestadual e internacional, exceto travessia
      '5022001', --Transporte por navegação interior de passageiros em linhas regulares, municipal, exceto travessia
      '5022002', --Transporte por navegação interior de passageiros em linhas regulares, intermunicipal, interestadual e internacional, exceto travessia
      '5030101', --Navegação de apoio marítimo
      '5030102', --Navegação de apoio portuário
      '5030103', --Serviço de rebocadores e empurradores
      '5091201', --Transporte por navegação de travessia, municipal
      '5091202', --Transporte por navegação de travessia intermunicipal, interestadual e internacional
      '5099801', --Transporte aquaviário para passeios turísticos
      '5099899' --Outros transportes aquaviários não especificados anteriormente
)


""" 

dados = bd.read_sql(query, billing_project_id="observatorio-portuario")   # Cada usuario deve setar o proprio project_id 

Downloading: 100%|██████████| 189482/189482 [00:34<00:00, 5487.10rows/s]


In [73]:
# Salvar o DataFrame em um arquivo CSV
dados.to_csv('caged.csv', encoding='utf-8', index=False)

# Confirmar que o arquivo foi salvo
print("DataFrame salvo em 'dados.csv'")

TypeError: __init__() got an unexpected keyword argument 'line_terminator'

In [72]:
num_valores_unicos = dados['sexo'].nunique()
num_valores_unicos

2

In [ ]:
dados = pd.read_csv("caged.csv", encoding = 'latin')

In [ ]:
#dados.backup = dados
dados_teste = dados 

In [ ]:
# verificar os dados 
dados.head(5)

In [ ]:
dados['ano_mes'] = dados['ano'].map(str) + '-' + dados['mes'].map(str) + '-01'
dados.tail(5)

In [ ]:
from datetime import datetime
import locale

locale.setlocale(locale.LC_ALL, 'pt_BR')
formato = "%Y-%m-%d"
dados.ano_mes = dados.ano_mes.apply(lambda linha: datetime.strptime(linha, formato))

In [ ]:
#Vamos fazer o label encoding com a criação de dicionário para recodificar variavel 
# https://www.sharpsightlabs.com/blog/recode-categorical-variable-python/ 
#dados.assign(teste  = dados.sexo.map(sexo_mapping)) #Usar essa abordagem para testar antes de salvar diretamente no dataframe


In [76]:
sexo_mapping = {1:'Masculino',
                3:'Feminino', 
               }

dados['sexo'] = dados['sexo'].map(sexo_mapping) 

dados.sexo.value_counts()


Series([], Name: sexo, dtype: int64)

In [77]:
dados.sexo

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
189477    NaN
189478    NaN
189479    NaN
189480    NaN
189481    NaN
Name: sexo, Length: 189482, dtype: object

In [ ]:
dados['sexo'] = dados['sexo'].map(sexo_mapping) 

dados.sexo.value_counts()

In [ ]:
saldo_mapping = {1:'Admissão', 
                -1:'Demissão', 
                }


dados['saldo_movimentacao'] = dados['saldo_movimentacao'].map(saldo_mapping)

# Isso informa apenas o número de registros
#dados.saldo_movimentacao.value_counts()


In [ ]:
# Com o datasaset completo, criar gráfico de linha com o saldo do período 

In [ ]:
intermitente_mapping = {1:'Sim', 
                0:'Nao', 
                9:'nan'
                    }

dados['indicador_trabalho_intermitente'] = dados['indicador_trabalho_intermitente'].map(intermitente_mapping)

#dados.indicador_trabalho_intermitente.value_counts()

In [ ]:
grauinstrucao_mapping = {1:'Analfabeto',
2:'Até 5ª Incompleto',
3:'5ª Completo Fundamental',
4:'6ª a 9ª Fundamental',
5:'Fundamental Completo',
6:'Médio Incompleto',
7:'Médio Completo',
8:'Superior Incompleto',
9:'Superior Completo',
10:'Mestrado',
11:'Doutorado',
80:'Pós-Graduação completa',
99:'Não Identificado'

}

dados['grau_instrucao'] = dados['grau_instrucao'].map(grauinstrucao_mapping)

#dados.grau_instrucao.value_counts()

In [ ]:
parcial_mapping = {1:'Sim', 
                0:'Nao', 
                9:'nan'
                    }

dados['indicador_trabalho_parcial'] = dados['indicador_trabalho_parcial'].map(parcial_mapping)

#dados.indicador_trabalho_parcial.value_counts()


In [ ]:
regiao_mapping = {1:'Norte',
                2:'Nordeste', 
                3:'Sudeste', 
                4:'Sul', 
                5:'Centro-Oeste', 
                9: 'nan'
                    }


In [ ]:
uf_mapping = {
    11:'Rondonia',
12:'Acre',
13:'Amazonas',
14:'Roraima',
15:'Para',
16:'Amapa',
17:'Tocantins',
21:'Maranhao',
22:'Piaui',
23:'Ceara',
24:'Rio-Grande-do-Norte',
25:'Paraiba',
26:'Pernambuco',
27:'Alagoas',
28:'Sergipe',
29:'Bahia',
31:'Minas-gerais',
32:'Espirito-Santo',
33:'Rio-de-Janeiro', 
35:'Sao-Paulo',
41:'Parana',
42:'Santa Catarina',
43:'Rio Grande do Sul',
50:'Mato Grosso do Sul',
51:'Mato Grosso',
52:'Goias',
53:'Distrito-Federal',
99: 'nan'

}

In [ ]:
secao_mapping = {
    
'A':'Agricultura, Pecuária, Produção Florestal, Pesca e Aquicultura',
'B':'Indústrias Extrativas',
'C':'Indústrias de Transformação',
'D':'Eletricidade e Gás',
'E':'Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação',
'F':'Construção',
'G':'Comércio, Reparação de Veículos Automotores e Motocicletas',
'H':'Transporte, Armazenagem e Correio',
'I':'Alojamento e Alimentação',
'J':'Informação e Comunicação',
'K':'Atividades Financeiras, de Seguros e Serviços Relacionados',
'L':'Atividades Imobiliárias',
'M':'Atividades Profissionais, Científicas e Técnicas',
'N':'Atividades Administrativas e Serviços Complementares',
'O':'Administração Pública, Defesa e Seguridade Social',
'P':'Educação',
'Q':'Saúde Humana e Serviços Sociais',
'R':'Artes, Cultura, Esporte e Recreação',
'S':'Outras Atividades de Serviços',
'T':'Serviços Domésticos',
'U':'Organismos Internacionais e Outras Instituições Extraterritoriais',
'Z':'Não identificado'
}

dados['cnae_2_secao'] = dados['cnae_2_secao'].map(secao_mapping)

dados.cnae_2_secao.value_counts()


In [ ]:
dados.tail(3)

In [ ]:
dados.cnae_2_subclasse.value_counts()

#Fazer o mapping aqui também 


In [ ]:
teste = dados
teste.tail(2)

In [ ]:
# Alguns dados do contexto nacional 

#teste.saldo_movimentacao.value_counts().plot(kind="bar")

# group by a column and count a variable and plot it
# teste.groupby('uf').size().plot(kind="bar")

dados[['sigla_uf', 'saldo_movimentacao']].groupby(['sigla_uf']).agg(['count'])



## Selecionar mês vigente para análise

In [ ]:
print(dados.ano_mes.min())
print(dados.ano_mes.max())

In [ ]:
# Esse método é manual. Tem que automatizar

saldo_mes_atual = dados.loc[(dados.ano_mes == "2024-06-01")]
saldo_mes_atual

In [ ]:
# sns.catplot(data=dados, x="sexo", kind="count", palette="ch:.25")
saldo_atual = sns.catplot(
    data=saldo_mes_atual, y="saldo_movimentacao", hue="sexo", kind="count",
    palette="pastel", edgecolor=".6"
)

# Verificar se os containers possuem dados antes de adicionar os rótulos
if saldo_atual.axes[0,0].containers:
    saldo_atual.axes[0,0].bar_label(saldo_atual.axes[0,0].containers[0])
    saldo_atual.axes[0,0].bar_label(saldo_atual.axes[0,0].containers[1])

# saldo_atual.fig.suptitle('Movimentação de vínculos de emprego no mês', fontsize=18)
saldo_atual.set_xlabels("Nº de registros")
saldo_atual.set_ylabels("")

In [ ]:
#sns.catplot(data=dados, x="sexo", kind="count", palette="ch:.25")

saldo_atual = sns.catplot(
    data=saldo_mes_atual, y="saldo_movimentacao", hue="sexo", kind="count",
    palette="pastel", edgecolor=".6",
)

# O objeto criado pela catplot não é uma barplot. Ver aqui: https://stackoverflow.com/questions/69810130/seaborn-display-values-on-barplot
saldo_atual.axes[0,0].bar_label(saldo_atual.axes[0,0].containers[0])
saldo_atual.axes[0,0].bar_label(saldo_atual.axes[0,0].containers[1])
#saldo_atual.fig.suptitle('Movimentação de vínculos de emprego no mês', fontsize=18)
saldo_atual.set_xlabels("Nº de registros")
saldo_atual.set_ylabels("")


In [ ]:
descending_order = saldo_mes_atual["grau_instrucao"].value_counts().sort_values(ascending=False).index

edu = sns.countplot(data=saldo_mes_atual, y="grau_instrucao", order=descending_order, hue = "saldo_movimentacao")

#ax.bar_label(ax.containers[0], label_type='edge')
container = edu.containers[0]
container_1 = edu.containers[1]
edu.bar_label(container, labels=[f'{x:,.0f}' for x in container.datavalues])
edu.bar_label(container_1, labels=[f'{x:,.0f}' for x in container_1.datavalues])
edu.legend(loc='upper right', bbox_to_anchor=(1.3, 1))


# Add some text for labels, title and custom x-axis tick labels, etc.
edu.set_ylabel(' ')
edu.set_xlabel('Registros')
edu.set_title('Movimentação de vínculos por escolaridade', fontsize=18)

# https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py 

In [ ]:
ax = sns.countplot(x="indicador_trabalho_parcial", hue="saldo_movimentacao", data=saldo_mes_atual)

#Adicionar label com casa decimal. Verificar aqui: https://stackoverflow.com/questions/70515542/adding-comma-to-bar-labels 

#ax.bar_label(ax.containers[0], label_type='edge')
container = ax.containers[0]
ax.bar_label(container, labels=[f'{x:,.0f}' for x in container.datavalues])

#https://www.kaggle.com/code/slamnz/seaborn-tutorial-count-plots
#https://medium.com/swlh/quick-guide-to-labelling-data-for-common-seaborn-plots-736e10bf14a9 

ax.set_ylabel('')
ax.set_xlabel('')
ax.set_title('Movimentação do Trabalho parcial', fontsize=18)

In [ ]:
plot_intermitente = sns.countplot(x="indicador_trabalho_intermitente", data=dados)

plot_intermitente.bar_label(plot_intermitente.containers[0], label_type='edge')

In [ ]:
pip install tabulate
#https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd

In [ ]:
from tabulate import tabulate


In [ ]:
intermitente = dados.indicador_trabalho_intermitente.value_counts()



In [ ]:
print(tabulate(dados, tablefmt='fancy_grid'))


In [ ]:
x = dados.groupby(['ano_mes', 'saldo_movimentacao']).size()

x

In [ ]:
x["difere"] = x["station_london"]-x["saldo_movimentacao"=="Demissão"]

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:

from matplotlib.pyplot import xticks

lineplot = sns.lineplot(data=dados, x="ano_mes", y="saldo_movimentacao", ci=None)
#xticks(rotation=40)


lineplot.set_xticklabels(lineplot.get_xticklabels(),rotation=30)
lineplot.set_xlabel("Mês")
lineplot.set_ylabel("Saldo")


#https://stackoverflow.com/questions/44954123/rotate-xtick-labels-in-seaborn-boxplot 
#https://stackoverflow.com/questions/45201514/how-to-edit-a-seaborn-legend-title-and-labels-for-figure-level-functions


In [ ]:
fig = sns.lineplot(
    data=dados.groupby(['ano_mes','saldo_movimentacao', 'sexo']).size().reset_index(name='count'), 
    x='ano_mes', y='count', hue = 'saldo_movimentacao', marker='o', ci=None)

fig.set_xticklabels(lineplot.get_xticklabels(),rotation=30)


In [ ]:
data=dados.groupby(['ano_mes','saldo_movimentacao', 'sexo']).size().reset_index(name='count')
data

In [ ]:
data

In [ ]:
plot = sns.lineplot(x='ano_mes', y='count', hue='saldo_movimentacao', ci=None, 
             data=data)
plot.set_xticklabels(lineplot.get_xticklabels(),rotation=30)

In [ ]:
ax1 = sns.catplot(
    data=newdf, y="saldo_movimentacao", hue="sexo", kind="count",
    palette="pastel", edgecolor=".6",
)

# O objeto criado pela catplot não é uma barplot. Ver aqui: https://stackoverflow.com/questions/69810130/seaborn-display-values-on-barplot
ax1.axes[0,0].bar_label(ax1.axes[0,0].containers[0])
ax1.axes[0,0].bar_label(ax1.axes[0,0].containers[1])


# https://stackoverflow.com/questions/55586912/seaborn-catplot-set-values-over-the-bars 
#https://stackoverflow.com/questions/28931224/how-to-add-value-labels-on-a-bar-chart/67561982#67561982
